## Sentiment Analysis

In [ ]:
# importing libraries 
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import contractions
from unidecode import unidecode
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
from autocorrect import Speller
import pandas as pd 
from gensim.models import Word2Vec
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
data = pd.read_csv("Train.csv")
data.head(10)

In [ ]:
# wordcloud  : ""
def generate_wordcloud(data,column):
    df = data[column].str.cat(sep=" ") # record1   record 2 
    text = " ".join([word for word in df.split()])# 
    word_cloud = WordCloud(height=500,width=700,background_color='white',min_font_size=10).generate(text)
    plt.figure(figsize=(10,16))
    plt.imshow(word_cloud)
    plt.axis("off")
    plt.show()

In [ ]:
generate_wordcloud(data,'text')

In [ ]:
# 1. remove blanklines,whitespaces,tabs
# 2. contraction mapping 
# 3. Handling accented characters
# 4. Cleaning : tokenization,remove punctuation,normalization,stopwords removal,remove numerical format,len(word)<2 removed
# 5. autocorrect 
# 6. stemming and lemmatization 

In [ ]:
# 1. remove blanklines,whitespaces,tabs 
def remove_spaces(data):
    "Function will remove all spaces"
    formatted_text = data.replace('\\n',' ').replace('\t',' ').replace("\\",' ')
    return formatted_text 
# 2. Contraction mapping 
def expand_text(data):
    fixed_text = contractions.fix(data)
    return fixed_text

# 3. Handling accented 
def handling_accented(data):
    fixed_text = unidecode(data)
    return fixed_text
# 4.cleaning 
stopword_list = stopwords.words("english")
stopword_list.remove("not")
stopword_list.remove("nor")
stopword_list.remove("no")
def cleaning(data):
    tokens = word_tokenize(data) # tokenization
    clean_text = [word.lower() for word in tokens if (word not in stopword_list) and(word not in punctuation) and(len(word)>2) and(word.isalpha())]
    return clean_text

# 5. autocorrect 
def autocorrection(data):
    spell = Speller(lang='en')
    corrected_text = spell(data)
    return corrected_text

# 6. Lemmatization 
def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_text.append(lemmatized_word)
    return " ".join(final_text)

In [ ]:
# data leakage 
x_train,x_test,y_train,y_test = train_test_split(data.text,data.label,test_size=0.25,random_state=45)

In [ ]:
# preprocessing 
clean_text_train = x_train.apply(remove_spaces)
clean_text_test = x_test.apply(remove_spaces)

clean_text_train = clean_text_train.apply(expand_text)
clean_text_test = clean_text_test.apply(expand_text)

clean_text_train = clean_text_train.apply(handling_accented)
clean_text_test = clean_text_test.apply(handling_accented)

clean_text_train = clean_text_train.apply(cleaning)
clean_text_test = clean_text_test.apply(cleaning)

clean_text_train = clean_text_train.apply(lemmatization)
clean_text_test = clean_text_test.apply(lemmatization)

In [ ]:
def filter_2(data):
    tokens = data.split()
    clean_text = [word for word in tokens if word not in stopword_list]
    return " ".join(clean_text)
clean_text_train = clean_text_train.apply(filter_2)
clean_text_test = clean_text_test.apply(filter_2)

In [ ]:
# implememtation of ngrams 
from nltk.util import ngrams
for unigrams in ngrams(clean_text_train[11449].split(),2):
    print(" ".join(unigrams))

In [ ]:
# Count vectorizer 
count = CountVectorizer(max_df = 0.95,max_features=1000)
count_val_train =count.fit_transform(clean_text_train)
count_val_test = count.transform(clean_text_test)

In [ ]:
len(count.get_feature_names())

In [ ]:
pd.DataFrame(count_val_train.A,columns=count.get_feature_names())

In [ ]:
count_mnb = MultinomialNB()
count_mnb.fit(count_val_train.A,y_train) 
predict_mnb = count_mnb.predict(count_val_test.A)
accuracy_count = accuracy_score(y_test,predict_mnb)*100
accuracy_count

In [ ]:
# TFIDF 
tfidf = TfidfVectorizer(max_df = 0.95,max_features=1000)
tfidf_train = tfidf.fit_transform(clean_text_train)
tfidf_test = tfidf.transform(clean_text_test)

In [ ]:
len(tfidf.get_feature_names())

In [ ]:
pd.DataFrame(tfidf_train.A,columns=tfidf.get_feature_names())

In [ ]:
tfidf_mnb = MultinomialNB()
tfidf_mnb.fit(tfidf_train.A,y_train)
predict_tfidf = tfidf_mnb.predict(tfidf_test.A)
accuracy_tfidf = accuracy_score(y_test,predict_tfidf)*100
print(f" accuracy_tfidf {accuracy_tfidf}")
print(f" accuracy_count {accuracy_count}")